In [ ]:
# %%
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

# %% [markdown]
# # Contents
# 1. Download cleaned LFS data
# 2. Group samples into categories
# 3. Classify employment status
# 4. Get main statistics
# 5. Loop over multiple datasets

# %% [markdown]
# # 1. Download cleaned LFS data

# %% [markdown]
# Get quarterly data

# %%
def get_LFS_data(year, quarter):
    year = int(year) % 100
    try:
        df = pd.read_csv(f'data_csv_cleaned\\LFS_{year}q{quarter}.csv', encoding='utf-8', dtype=str)
        # Replace empty strings with NaN
        df = df.replace(r'^\s*$', np.nan, regex=True)
        # numeric columns to float
        numeric_columns = ['main_hr', 'part_time_hr', 'total_hr', 'wage_per_month', 'wage_bonus', 'wage_ot', 'other_income', 'weight', 'age']
        for col in numeric_columns:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce')
                
    except FileNotFoundError:
        print(f"File LFS_{year}q{quarter}.csv not found.")
        return None
    
    return df

# %%
#df = get_LFS_data(year=2566, quarter=1)
#df

# %% [markdown]
# Get half-year data

# %%
def get_LFS_data_h(year, half: int):
    
    quarter_to_half_year = {
        1: [1, 2],
        2: [3, 4]
    }

    df = pd.DataFrame()
    for quarter in quarter_to_half_year[half]:
        try:
            df = pd.concat([df, get_LFS_data(year, quarter)])
        
        except Exception as e:
            print(f"{year}q{quarter} -- error occurred: {e}")
            break
    
    return df

# %%
# df_67_h1 = get_LFS_data_h(year=2567, half=1)
# df_67_h1

# %%
def get_LFS_data_y(year: int):

    quarters = [1, 2, 3, 4]

    df = pd.DataFrame()
    for quarter in quarters:
        try:
            df = pd.concat([df, get_LFS_data(year, quarter)])
        
        except Exception as e:
            print(f"{year} -- error occurred: {e}")
            break
    
    return df

# %%
#df_66 = get_LFS_data_y(year=2566)
#df_66

# %% [markdown]
# # 2. Group samples into categories

# %% [markdown]
# ## Group by production category (A - U) and TSIC

# %%
def get_TSIC_category(df):
    TSIC_to_CAT = pd.read_csv('TSIC\\TSIC_to_Category.csv', encoding='utf-8', dtype=str)
    TSIC_to_CAT['TSIC_2'] = TSIC_to_CAT['TSIC_2'].astype(int)
    TSIC_to_CAT['TSIC_2'] = TSIC_to_CAT['TSIC_2'].astype(str)
    df['TSIC'] = df['industry'].str.slice(0, -3)
    df['TSIC'] = pd.to_numeric(df['TSIC'], errors='coerce').astype('Int64')
    df['TSIC'] = df['TSIC'].astype(str)
    # in df['TSIC'], change pd.NA to np.nan
    df['TSIC'] = df['TSIC'].replace({'<NA>': np.nan})
    df = df.merge(TSIC_to_CAT, left_on='TSIC', right_on='TSIC_2', how='left')
    df.drop(columns=['TSIC_2'], inplace=True)
    # rename 'Category' to 'category'
    df.rename(columns={'Category': 'category'}, inplace=True)

    return df

# %%
#df = get_TSIC_category(df)

# %%
#df[['TSIC', 'category']]

# %% [markdown]
# ## Group by Educational Level

# %%
def get_education(df):
    
    class_edu = {
        'Lower than High School': [0, 1, 2, 3],
        'High School': [4],
        'Higher Education': [5, 6, 7, 8],
    }

    df['edu_level_1st_digit'] = df['edu_level'].str.slice(0, 1)
    # change to numeric
    df['edu_level_1st_digit'] = pd.to_numeric(df['edu_level_1st_digit'], errors='coerce').astype('Int64')
    # map to class_edu dictionary
    df['education'] = df['edu_level_1st_digit'].map({v: k for k, values in class_edu.items() for v in values})
    # drop edu_level_1st_digit
    df.drop(columns=['edu_level_1st_digit'], inplace=True)
    
    return df

# %%
#df = get_education(df)

# %%
#df[['edu_level', 'education']]

# %%
def get_education_new(df):
    
    class_edu = {
        'Lower or equal than High School': [0, 1, 2, 3, 4],
        'Higher Education': [5, 6, 7, 8],
    }

    df['edu_level_1st_digit'] = df['edu_level'].str.slice(0, 1)
    # change to numeric
    df['edu_level_1st_digit'] = pd.to_numeric(df['edu_level_1st_digit'], errors='coerce').astype('Int64')
    # map to class_edu dictionary
    df['education_new'] = df['edu_level_1st_digit'].map({v: k for k, values in class_edu.items() for v in values})
    # drop edu_level_1st_digit
    df.drop(columns=['edu_level_1st_digit'], inplace=True)
    
    return df

# %% [markdown]
# # 3. Classify employment status

# %%
def classify_emp_status(df):
    class_emp = {'1': 'Employed',
                 '2': 'Seasonally Unemployed',
                 '3': 'Unemployed',
                 '4': 'Out of Labor Force',}

    df['employ'] = '4'
    df.loc[df['wk_7day'] == '1', 'employ'] = '1'
    df.loc[df['receive'] == '1', 'employ'] = '1'
    df.loc[df['return'] == '1', 'employ'] = '1'
    df.loc[df['re_unavailable'] == '3', 'employ'] = '2'
    df.loc[df['seeking'] == '1', 'employ'] = '3'
    df.loc[df['available'] == '1', 'employ'] = '3'

    # drop unnecessary columns
    df.drop(columns=['wk_7day', 'receive', 'return', 're_unavailable', 'seeking', 'available'], inplace=True)
    
    return df, class_emp

# %%
#df, class_emp = classify_emp_status(df)

# %%
#df[['wk_7day', 'receive', 'return', 're_unavailable', 'seeking', 'available', 'employ']]

# %% [markdown]
# # 4. Get main statistics

# %% [markdown]
# ## Main statistics without grouping

# %%
def get_summary(df, group_by=None):
    if group_by is None or group_by == []:
        dfs = [(None, df)]
    else:
        dfs = list(df.groupby(group_by))

    summary_list = []
    for group_key, group_df in dfs:
        employ_status_sum = group_df.groupby(['employ'])['weight'].sum().to_frame().reset_index()
        labor_force = employ_status_sum[employ_status_sum['employ'].isin(['1', '2', '3'])]['weight'].sum()
        # handle if there is no unemployed in group
        if '3' in employ_status_sum['employ'].values:
            unemp_rate = employ_status_sum.loc[employ_status_sum['employ'] == '3', 'weight'].values[0] / labor_force if labor_force > 0 else np.nan
        else:
            unemp_rate = np.nan
        working_age = group_df[group_df['age'] >= 15]['weight'].sum()
        lfpr = labor_force / working_age if working_age > 0 else np.nan
        employed = group_df[group_df['employ'].isin(['1'])]  # employed
        avg_work_hr = (employed['total_hr'] * employed['weight']).sum() / employed['weight'].sum() if employed['weight'].sum() > 0 else np.nan
        avg_wage_per_month = (employed['wage_per_month'] * employed['weight']).sum() / employed['weight'].sum() if employed['weight'].sum() > 0 else np.nan
        employed = employed.copy()
        employed['wage_bonus_monthly'] = employed['wage_bonus'] / 12
        employed['total_income'] = employed['wage_per_month'] + employed['wage_ot'] + employed['wage_bonus_monthly']
        avg_total_income = (employed['total_income'] * employed['weight']).sum() / employed['weight'].sum() if employed['weight'].sum() > 0 else np.nan

        summary = {
            # group keys
            **({group_by[i]: group_key[i] for i in range(len(group_by))} if group_by else {}),
            'Employed': employ_status_sum.loc[employ_status_sum['employ'] == '1', 'weight'].values[0].round(0) if '1' in employ_status_sum['employ'].values else 0,
            'Seasonally Unemployed': employ_status_sum.loc[employ_status_sum['employ'] == '2', 'weight'].values[0].round(0) if '2' in employ_status_sum['employ'].values else 0,
            'Unemployed': employ_status_sum.loc[employ_status_sum['employ'] == '3', 'weight'].values[0].round(0) if '3' in employ_status_sum['employ'].values else 0,
            'Labor Force': labor_force.round(0) if not pd.isna(labor_force) else 0,
            'Out of Labor Force': employ_status_sum.loc[employ_status_sum['employ'] == '4', 'weight'].values[0].round(0) if '4' in employ_status_sum['employ'].values else 0,
            'Unemployment Rate': (unemp_rate * 100).round(2) if not pd.isna(unemp_rate) else np.nan,
            'Labor Force Participation Rate': (lfpr * 100).round(2) if not pd.isna(lfpr) else np.nan,
            'Average Working Hours': avg_work_hr.round(2) if not pd.isna(avg_work_hr) else np.nan,
            'Average Wage per Month': avg_wage_per_month.round(2) if not pd.isna(avg_wage_per_month) else np.nan,
            'Average Total Income': avg_total_income.round(2) if not pd.isna(avg_total_income) else np.nan,
        }
        summary_list.append(summary)

    summary = pd.DataFrame(summary_list)
    
    return summary

# %%
#summary = get_summary(df)

# %%
#summary

# %% [markdown]
# ## Main Statistics with grouping

# %%
def get_group_summary(df, group_by):
    # filter employed people
    employed = df[df['employ'].isin(['1'])] # employed

    # find employment sum for each TSIC category
    employed_sum = employed.groupby(group_by)['weight'].sum().to_frame().reset_index()
    employed_sum.rename(columns={'weight': 'employment'}, inplace=True)
    employed_sum['employment'] = employed_sum['employment'].round(0)

    # find average working hours for each TSIC category
    avg_work_hr = employed.groupby(group_by)['total_hr'].mean().reset_index()

    # replace missing values for wage_per_month, wage_bonus, wage_ot with 0
    employed_dropna = employed.dropna(subset=['wage_per_month', 'wage_per_month', 'wage_bonus', 'wage_ot'])
    # find average wage per month for each TSIC category
    avg_wage_per_month = employed_dropna.groupby(group_by)['wage_per_month'].mean().reset_index()

    # find average total income for each category
    employed_dropna['wage_bonus_monthly'] = employed_dropna['wage_bonus'] / 12
    employed_dropna['total_income'] = employed_dropna['wage_per_month'] + employed_dropna['wage_ot'] + employed_dropna['wage_bonus_monthly']
    avg_total_income = employed_dropna.groupby(group_by)['total_income'].mean().reset_index()

    # merge all dataframes
    df_summary = employed_sum.merge(avg_work_hr, on=group_by, how='left')
    df_summary = df_summary.merge(avg_wage_per_month, on=group_by, how='left')
    df_summary = df_summary.merge(avg_total_income, on=group_by, how='left')

    # round 'total_hr', wage_per_month, and 'total_income' to 2 decimal places
    df_summary['total_hr'] = df_summary['total_hr'].round(2)
    df_summary['wage_per_month'] = df_summary['wage_per_month'].round(2)
    df_summary['total_income'] = df_summary['total_income'].round(2)

    return df_summary

# %%
#group_by = ['TSIC','education']
#education_group_summary = get_group_summary(df, group_by) # education

# %%
#education_group_summary

# %% [markdown]
# # 5. Loop over multiple datasets

# %%
def get_summary_table(start_year, end_year=None, freq='q', group_by=None):
    '''
    Get summary table for the specified year range and frequency.

    Parameters:
    - start_year (int): starting year
    - end_year (int): ending year
    - freq (str): 'q' for quarterly, 'h' for half-yearly
    - group_by (str): 'TSIC', 'category', 'education'

    Returns:
    - summary_table (DataFrame): summary table
    '''
    if end_year is None:
        end_year = start_year
    summary_table = pd.DataFrame()
    for year in range(start_year, end_year+1):
        if freq == 'q':
            for quarter in range(1, 5):
                try:
                    df = get_LFS_data(year, quarter)
                    df = get_TSIC_category(df)
                    df = get_education(df)
                    df = get_education_new(df)
                    df, class_emp = classify_emp_status(df)
                    if group_by is not None:
                        summary = get_group_summary(df, group_by)
                    else:
                        summary = get_summary(df)
                    summary['Year'] = year
                    summary['Quarter'] = quarter
                    summary_table = pd.concat([summary_table, summary], ignore_index=True)
                except:
                    pass

        elif freq == 'h':
            for half in range(1, 3):
                try:
                    df = get_LFS_data_h(year, half)
                    df = get_TSIC_category(df)
                    df = get_education(df)
                    df = get_education_new(df)
                    df, class_emp = classify_emp_status(df)
                    if group_by is not None:
                        summary = get_group_summary(df, group_by)
                    else:
                        summary = get_summary(df)
                    summary['Year'] = year
                    summary['Half'] = half
                    summary_table = pd.concat([summary_table, summary], ignore_index=True)
                except:
                    pass
                
    # reordering columns, putting 'Year' and 'Quarter' or 'Half' at the beginning
    cols = summary_table.columns.tolist()
    if freq == 'q':
        cols = cols[-2:] + cols[:-2]
    elif freq == 'h':
        cols = cols[-2:] + cols[:-2]
    summary_table = summary_table[cols]
    
    return summary_table

# %% [markdown]
# ## Get the date from 2564 - 2567 on half-year basis

# %%
#start_year = 2566
#end_year = 2566
#freq = 'q'

# %% [markdown]
# ## Get a summary table

# %%
#summary_table = get_summary_table(start_year, end_year, freq)
#summary_table

# %%
#summary_table.to_csv(f'summary_tables\\summary_table_{start_year}_{end_year}_{freq}.csv', index=False)

# %% [markdown]
# ## Get a summary table by production category

# %%
#summary_table_category = get_summary_table(start_year, end_year, freq, group_by='category')
#summary_table_category

# %%
#summary_table_category.to_csv(f'summary_tables\\summary_table_category_{start_year}_{end_year}_{freq}.csv', index=False)

# %% [markdown]
# ## Get a summary table by TSIC

# %%
#summary_table_TSIC = get_summary_table(start_year, end_year, freq, group_by='TSIC')
#summary_table_TSIC

# %%
#summary_table_TSIC.to_csv(f'summary_tables\\summary_table_TSIC_{start_year}_{end_year}_{freq}.csv', index=False)

# %% [markdown]
# ## Get a summary table by education 

# %%
#summary_table_education = get_summary_table(start_year, end_year, freq, group_by='education')
#summary_table_education

# %%
#summary_table_education.to_csv(f'summary_tables\\summary_table_education_{start_year}_{end_year}_{freq}.csv', index=False)

# %% [markdown]
# ## Get a summary table by TSIC and education 

# %%
#group_by = ['TSIC', 'education']

# %%
#summary_table_TSIC_education = get_summary_table(start_year, end_year, freq, group_by)
#summary_table_TSIC_education

# %%
#summary_table_TSIC_education.to_csv(f'summary_tables\\summary_table_TSIC_education_{start_year}_{end_year}_{freq}.csv', index=False)

# %% [markdown]
# ## Get a summay table by TSIC and new_education

# %%
#group_by = ['TSIC', 'education_new']

# %%
#summary_table_TSIC_education_new = get_summary_table(start_year, end_year, freq, group_by)
#summary_table_TSIC_education_new

# %%
#summary_table_TSIC_education_new.to_csv(f'summary_tables\\summary_table_TSIC_education_new_{start_year}_{end_year}_{freq}.csv', index=False)

# %%
# export this file as python script
#!jupyter nbconvert --to script LFS_data_analysis_modified.ipynb
